In [2]:
import requests
base_url = 'http://localhost:7999'

In [3]:
class Session(requests.Session):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.headers.update({'Content-Type': 'application/json'})
    
    def get(self, url, *args, **kwargs):
        return super().get(base_url + url, *args, **kwargs)
    
    def post(self, url, *args, **kwargs):
        return super().post(base_url + url, *args, **kwargs)
    
    def patch(self, url, *args, **kwargs):
        return super().patch(base_url + url, *args, **kwargs)
    
    def delete(self, url, *args, **kwargs):
        return super().delete(base_url + url, *args, **kwargs)

client = Session()
resp = client.get("/ping")
resp.json()

{'data': 'pong!'}

In [4]:
import os
import sys

backend_path = os.path.abspath(os.path.join(os.getcwd(), '../backend'))
sys.path.append(backend_path)
from app.internal.firebase import db

In [53]:
query = "Nip IND ETF Junior BeES".upper()
print(query)


docs = await db.collection("tickers").where("name", ">=", query).where("name", "<=", query + '\uf8ff').get()
# doc = await db.collection("tickers").where("ticker", "<=", query).where("ticker", ">=", query + "~").get()

if docs:
    print(docs[0].to_dict())

print(docs)

NIP IND ETF JUNIOR BEES
{'created_at': '2024-05-05T12:20:40.104545Z', 'exchange': 'NSE', 'lot_size': 1, 'upstox_instrument_key': 'NSE_EQ|INF732E01045', 'exchange_token': 10939, 'updated_at': '2024-05-05T12:20:40.104547Z', 'ticker': 'JUNIORBEES', 'instrument_type': 'EQUITY', 'name': 'NIP IND ETF JUNIOR BEES'}


/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_8262/3214706254.py:5: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  docs = await db.collection("tickers").where("name", ">=", query).where("name", "<=", query + '\uf8ff').get()


In [61]:
from datetime import datetime
date = datetime.now()
print(date.strftime("%Y-%m-%d"))

2024-05-05


In [62]:
def format_date_path(date: datetime) -> str:
    return date.strftime("%Y-%m-%d")

In [5]:
from datetime import datetime
from typing import Dict, Optional, Annotated
from pydantic import BaseModel, Field, validator, StringConstraints

def format_date_path(date: datetime) -> str:
    try:
        return date.strftime("%Y-%m-%d")
    except:
        raise ValueError(f"Invalid date format {date}")
        
def validate_date_path(date: str):
    try:
        datetime.strptime(date, "%Y-%m-%d")
    except:
        raise ValueError(f"Invalid date format {date}")

class CandleBase(BaseModel):
    open: float
    high: float
    low: float
    close: float
    volume: int
    open_interest: Optional[int] = None

class MonthlyCandleBase(BaseModel):
    month: Annotated[str, StringConstraints(pattern=r"^(0[1-9]|1[0-2])$")]
    year: Annotated[str, StringConstraints(pattern=r"^(19\d{2}|20\d{2})$")]
    daily: Dict[str, CandleBase] = Field(default_factory=dict)
    monthly: Optional[CandleBase] = None

    @validator("daily", pre=True, allow_reuse=True)
    def validate_daily(cls, v):
        for key in v:
            validate_date_path(key)
        return v

/var/folders/lq/280gf8ld14v161g6ngpdjjnc0000gn/T/ipykernel_52500/2471255407.py:31: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.6/migration/
  @validator("daily", pre=True, allow_reuse=True)


In [7]:
ticker = "100"
date_from = "2022-12-15"
date_to = "2023-12-12"
params = {"start_date": date_from, "end_date": date_to}
resp = client.get(f"/tickers/{ticker}/candles", params=params)

resp.json()

{'2022-12-15': {'open': 650.75,
  'high': 658.0,
  'low': 647.2,
  'close': 651.55,
  'volume': 377599,
  'open_interest': 0},
 '2022-12-16': {'open': 648.4,
  'high': 654.6,
  'low': 636.5,
  'close': 639.0,
  'volume': 581758,
  'open_interest': 0},
 '2022-12-19': {'open': 638.05,
  'high': 649.75,
  'low': 638.05,
  'close': 648.1,
  'volume': 369803,
  'open_interest': 0},
 '2022-12-20': {'open': 648.0,
  'high': 654.2,
  'low': 640.15,
  'close': 645.8,
  'volume': 387078,
  'open_interest': 0},
 '2022-12-21': {'open': 649.05,
  'high': 652.5,
  'low': 640.5,
  'close': 642.9,
  'volume': 572046,
  'open_interest': 0},
 '2022-12-22': {'open': 642.1,
  'high': 656.0,
  'low': 632.85,
  'close': 645.9,
  'volume': 920662,
  'open_interest': 0},
 '2022-12-23': {'open': 639.9,
  'high': 642.8,
  'low': 601.2,
  'close': 603.4,
  'volume': 1406744,
  'open_interest': 0},
 '2022-12-26': {'open': 597.1,
  'high': 621.4,
  'low': 583.0,
  'close': 615.65,
  'volume': 831894,
  'open_inter

In [4]:
date_from = "2024-01-01"
date_to = "2024-02-02"
date_from <= date_to

True

In [13]:
date = "2024-07-02"
id = "SCNM_0025"

resp = client.get(f"/smallcases/{id}/constituents?date={date}")
resp.json()

{'start_date': '2024-03-15',
 'end_date': '2024-05-09',
 'constituents': [{'smallcase_name': 'Tata Power Company Ltd',
   'original_weightage': 0.07,
   'kelly_weightage': None,
   'standard_deviation': None},
  {'smallcase_name': 'Exide Industries Ltd',
   'original_weightage': 0.08,
   'kelly_weightage': None,
   'standard_deviation': None},
  {'smallcase_name': 'Amara Raja Energy & Mobility Ltd',
   'original_weightage': 0.09,
   'kelly_weightage': None,
   'standard_deviation': None},
  {'smallcase_name': 'Sundram Fasteners Ltd',
   'original_weightage': 0.13,
   'kelly_weightage': None,
   'standard_deviation': None},
  {'smallcase_name': 'Sona BLW Precision Forgings Ltd',
   'original_weightage': 0.07,
   'kelly_weightage': None,
   'standard_deviation': None},
  {'smallcase_name': 'Samvardhana Motherson International Ltd',
   'original_weightage': 0.08,
   'kelly_weightage': None,
   'standard_deviation': None},
  {'smallcase_name': 'Endurance Technologies Ltd',
   'original_wei